# Description:
这个jupyter完成基于内容的推荐， 用的数据集依然是movieLens数据集。总体流程如下：<br>

 ①建立物品画像

- ①用户打tag ②电影的分类值
- 根据电影的id 把tag和分类值合并起来 求tf-idf
- 根据tf-idf的结果 为每一部电影筛选出 top-n（tf-idf比较大的）个关键词
- 电影id-关键词-关键词权重

② 建立倒排索引

- 通过关键词找到电影
- 遍历 电影id-关键词-关键词权重 数据， 读取每一个关键词，用关键词作为key [(关键词对应的电影id,tfidf)] 作为value 保存到dict当中

③ 用户画像

- 看用户看过那些电影， 到电影的 电影id-关键词-关键词权重 数据中 找到电影所对应的关键词  
- 把用户看过的所有的关键词放到一起 统计词频 每个词出现了几次
- 出现次数多的关键词 作为用户的兴趣词，这个兴趣词实际上就是用户画像的关键词

④ 根据用户的兴趣词 找到兴趣词对应的电影 多个兴趣词可能对应一个电影 {电影id：[关键词1权重，关键词2权重]}

- 把每一个部电影对应的关键词权重求和之后 排序  权重比较高的排在前面 推荐给用户

In [35]:
import numpy as np
import pandas as pd

# 物品画像

## 构建数据集

In [36]:
_tags = pd.read_csv("ml-latest-small/all-tags.csv", usecols=range(1, 3)).dropna()
tags = _tags.groupby("movieId").agg(list)

In [37]:
tags.head()

,tag
movieId,
1,"[animated, buddy movie, Cartoon, cgi, comedy, ..."
2,"[fantasy, adapted from:book, animals, bad cgi,..."
3,"[moldy, old, Ann Margaret, Burgess Meredith, D..."
4,"[characters, girl movie, characters, chick fli..."
5,"[steve martin, steve martin, pregnancy, remake..."


In [38]:
movies = pd.read_csv("ml-latest-small/movies.csv", index_col="movieId")
movies['genres'] = movies['genres'].apply(lambda x: x.split("|"))

In [39]:
movies.head()

,title,genres
movieId,,
1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
2,Jumanji (1995),"[Adventure, Children, Fantasy]"
3,Grumpier Old Men (1995),"[Comedy, Romance]"
4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
5,Father of the Bride Part II (1995),[Comedy]


In [40]:
movies_index = set(movies.index) & set(tags.index)
new_tags = tags.loc[list(movies_index)]
ret = movies.join(new_tags)

In [41]:
ret.head()

,title,genres,tag
movieId,,,
1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]","[animated, buddy movie, Cartoon, cgi, comedy, ..."
2,Jumanji (1995),"[Adventure, Children, Fantasy]","[fantasy, adapted from:book, animals, bad cgi,..."
3,Grumpier Old Men (1995),"[Comedy, Romance]","[moldy, old, Ann Margaret, Burgess Meredith, D..."
4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]","[characters, girl movie, characters, chick fli..."
5,Father of the Bride Part II (1995),[Comedy],"[steve martin, steve martin, pregnancy, remake..."


In [42]:
df = map(lambda x: (x[0], x[1], x[2], x[2]+x[3]) if x[3] is not np.nan else (x[0], x[1], x[2], []), ret.itertuples())

In [43]:
movies_dataset = pd.DataFrame(df, columns=['movieId', 'title', 'genres', 'tags'])

In [44]:
movies_dataset.head()

,movieId,title,genres,tags
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]","[Adventure, Animation, Children, Comedy, Fanta..."
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]","[Adventure, Children, Fantasy, fantasy, adapte..."
2,3,Grumpier Old Men (1995),"[Comedy, Romance]","[Comedy, Romance, moldy, old, Ann Margaret, Bu..."
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]","[Comedy, Drama, Romance, characters, girl movi..."
4,5,Father of the Bride Part II (1995),[Comedy],"[Comedy, steve martin, steve martin, pregnancy..."


In [45]:
def get_movie_dataset():
    # 加载基于所有电影的标签
    # all-tag.csv
    _tags = pd.read_csv("ml-latest-small/all-tags.csv", usecols=range(1, 3)).dropna()
    tags = _tags.groupby("movieId").agg(list)
    
    # 加载电影列表数据
    movies = pd.read_csv("ml-latest-small/movies.csv", index_col="movieId")
    # 将电影的类别词分开
    movies['genres'] = movies['genres'].apply(lambda x: x.split("|"))
    
    # 为每部电影匹配对应的标签数据， 如果没有将会是NAN
    movies_index = set(movies.index) & set(tags.index)
    new_tags = tags.loc[list(movies_index)]
    ret = movies.join(new_tags)
    
    # 构建电影数据集， 包含电影ID， 电影名称， 类别和标签四个字段
    # 如果电影没有标签数据， 就替换为空列表
    # map(fun, 可迭代对象)
    df = map(lambda x: (x[0], x[1], x[2], x[2]+x[3]) if x[3] is not np.nan else (x[0], x[1], x[2], []), ret.itertuples())
    movies_dataset = pd.DataFrame(df, columns=['movieId', 'title', 'genres', 'tags'])
    
    movies_dataset.set_index("movieId", inplace=True)
    return movies_dataset

In [46]:
movies_dataset = get_movie_dataset()

In [47]:
movies_dataset.head()

,title,genres,tags
movieId,,,
1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]","[Adventure, Animation, Children, Comedy, Fanta..."
2,Jumanji (1995),"[Adventure, Children, Fantasy]","[Adventure, Children, Fantasy, fantasy, adapte..."
3,Grumpier Old Men (1995),"[Comedy, Romance]","[Comedy, Romance, moldy, old, Ann Margaret, Bu..."
4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]","[Comedy, Drama, Romance, characters, girl movi..."
5,Father of the Bride Part II (1995),[Comedy],"[Comedy, steve martin, steve martin, pregnancy..."


## Tfidf Model

In [48]:
from gensim.models import TfidfModel
from pprint import pprint
from gensim.corpora import Dictionary

In [49]:
dataset = movies_dataset["tags"].values

In [50]:
# 根据数据集建立词袋， 并统计词频， 将所有词放入一个词典， 使用索引进行获取
dct = Dictionary(dataset)

In [51]:
# 对于每条数据， 返回对应的词索引和词频
corpus = [dct.doc2bow(line) for line in dataset]     # 这个东西统计每个单词在每部电影中出现的词频 tf

In [52]:
# 根据这个词频， 就可以训练Tf-IDF模型， 计算TF-IDF值
model = TfidfModel(corpus)

In [53]:
model[corpus[0]]

[(0, 0.011056904421060245),
 (1, 0.022544275919360972),
 (2, 0.007079278641715789),
 (3, 0.002798045765174126),
 (4, 0.012559237823372017),
 (5, 0.011503590086751218),
 (6, 0.003822990021240867),
 (7, 0.006436556765484401),
 (8, 0.012559237823372017),
 (9, 0.010358359169695198),
 (10, 0.007294227222687766),
 (11, 0.012559237823372017),
 (12, 0.003052808998969177),
 (13, 0.06290655524434838),
 (14, 0.0036895245437134857),
 (15, 0.0013298049743059352),
 (16, 0.010358359169695198),
 (17, 0.003681211552798544),
 (18, 0.2598207646720874),
 (19, 0.0070304347783136335),
 (20, 0.011611370980069282),
 (21, 0.0034440300273526517),
 (22, 0.008395905773059107),
 (23, 0.029694717526112635),
 (24, 0.004848884825145518),
 (25, 0.007448038929756369),
 (26, 0.006809393011219959),
 (27, 0.5233581922752641),
 (28, 0.02748351220336432),
 (29, 0.011056904421060245),
 (30, 0.05370223399240885),
 (31, 0.2552979527940718),
 (32, 0.012559237823372017),
 (33, 0.008395905773059107),
 (34, 0.0080522376164367),
 (

In [54]:
# 保存每个电影tf-idf值最高的30个标签
movie_profile = {}
for i, mid in enumerate(movies_dataset.index):
    # 对于每部电影， 返回每个标签的tf-idf值
    tfidf_vec = model[corpus[i]]
    # 按照tfidf值排序， 然后取Top-N
    movies_tags = sorted(tfidf_vec, key=lambda x: x[1], reverse=True)[:30]
    # 根据关键词提取对应的名称
    movie_profile[mid] = dict(map(lambda x: (dct[x[0]], x[1]), movies_tags))

In [55]:
pprint(movie_profile[2])

{'Adaptation of Book': 0.034075799351485185,
 'Children': 0.050052200548347164,
 'Chris Van Allsburg': 0.05999931871653004,
 'Filmed in BC': 0.05620872438935659,
 'For children': 0.028932283348594864,
 'Joe Johnston': 0.05272105069429941,
 'Kirsten Dunst': 0.110664625189357,
 'Lebbat': 0.03150404135004002,
 'Robin Williams': 0.695468714308112,
 'adapted from:book': 0.033531509912372595,
 'animals': 0.21493519083390042,
 'bad cgi': 0.1991963253408426,
 'board game': 0.3156946502624451,
 'childhood recaptured': 0.08999897807479505,
 'fantasy': 0.28312224495427457,
 'fiction': 0.044568770707859084,
 'game': 0.044568770707859084,
 'giant insect': 0.028104362194678295,
 'herds of CGI animals': 0.05999931871653004,
 'jungle': 0.037407363762693525,
 'kid flick': 0.20738815492035886,
 'magic board game': 0.15752020675020012,
 'monkey': 0.08193209561708079,
 'new home': 0.028104362194678295,
 'not for kids': 0.07136630203711364,
 'scary': 0.13475111914523905,
 'see also:Zathura': 0.034075799351

In [56]:
# 完善物品画像的提取
def create_movie_profile(movie_dataset):
    '''
    使用tfidf，分析提取topn关键词
    :param movie_dataset:
    :return:
    '''
    dataset = movie_dataset["tags"].values

    from gensim.corpora import Dictionary
    # 根据数据集建立词袋，并统计词频，将所有词放入一个词典，使用索引进行获取
    dct = Dictionary(dataset)
    # 根据将每条数据，返回对应的词索引和词频
    corpus = [dct.doc2bow(line) for line in dataset]
    # 训练TF-IDF模型，即计算TF-IDF值
    model = TfidfModel(corpus)

    _movie_profile = []
    for i, data in enumerate(movie_dataset.itertuples()):
        mid = data[0]
        title = data[1]
        genres = data[2]
        vector = model[corpus[i]]
        movie_tags = sorted(vector, key=lambda x: x[1], reverse=True)[:30]
        topN_tags_weights = dict(map(lambda x: (dct[x[0]], x[1]), movie_tags))
        # 将类别词的添加进去，并设置权重值为1.0
        for g in genres:
            topN_tags_weights[g] = 1.0
        topN_tags = [i[0] for i in topN_tags_weights.items()]
        _movie_profile.append((mid, title, topN_tags, topN_tags_weights))

    movie_profile = pd.DataFrame(_movie_profile, columns=["movieId", "title", "profile", "weights"])
    movie_profile.set_index("movieId", inplace=True)
    return movie_profile

In [57]:
movie_dataset = get_movie_dataset()
movie_profile = create_movie_profile(movie_dataset)

In [58]:
movie_profile.head()

,title,profile,weights
movieId,,,
1,Toy Story (1995),"[Pixar, pixar, animation, toys, Disney, Tom Ha...","{'Pixar': 0.5233581922752641, 'pixar': 0.43327..."
2,Jumanji (1995),"[Robin Williams, board game, time travel, fant...","{'Robin Williams': 0.695468714308112, 'board g..."
3,Grumpier Old Men (1995),"[moldy, Walter Matthau, Jack Lemmon, fishing, ...","{'moldy': 0.4257623617953107, 'Walter Matthau'..."
4,Waiting to Exhale (1995),"[chick flick, girl movie, characters, interrac...","{'chick flick': 0.6111589664927171, 'girl movi..."
5,Father of the Bride Part II (1995),"[steve martin, pregnancy, childhood classics, ...","{'steve martin': 0.7577224298058415, 'pregnanc..."


## 建立倒排索引

In [59]:
# 建立倒排索引  为了根据指定关键词迅速匹配到对应的电影，因此需要对物品画像的标签词，建立**倒排索引**
# 通常数据存储数据， 都是以物品的ID作为索引， 去提取物品的其他信息数据
# 而倒排索引就是用物品的其他数据作为索引， 去提取他们对应的物品的ID列表
def create_inverted_table(movie_profile):
    inverted_table = {}
    for mid, weights in movie_profile['weights'].iteritems():
        for tag, weight in weights.items():
            # 到inverted_table dict 用tag作为key去取值， 如果取不到就返回[]
            _ = inverted_table.get(tag, [])
            _.append((mid, weight))
            inverted_table.setdefault(tag, _)
    return inverted_table

In [60]:
inverted_table = create_inverted_table(movie_profile)

AttributeError: 'Series' object has no attribute 'iteritems'

In [27]:
pprint(inverted_table)         # 这样就可以直接根据标签去推荐电影了  具体是在建立完用户画像的时候用到

# 用户画像
构建步骤：
1. 根据用户的评分历史，结合物品画像，将有观影记录的电影的画像标签作为初始标签反打到用户身上
2. 通过对用户观影标签的次数进行统计，计算用户的每个初始标签的权重值，排序后选取TOP-N作为用户最终的画像标签

In [28]:
import collections
from functools import reduce

In [29]:
watch_record = pd.read_csv("ml-latest-small/ratings.csv", usecols=range(2), dtype={"userId":np.int32, "movieId": np.int32})
watch_record = watch_record.groupby("userId").agg(list)

In [30]:
watch_record.head()

,movieId
userId,
1,"[1, 3, 6, 47, 50, 70, 101, 110, 151, 157, 163,..."
2,"[318, 333, 1704, 3578, 6874, 8798, 46970, 4851..."
3,"[31, 527, 647, 688, 720, 849, 914, 1093, 1124,..."
4,"[21, 32, 45, 47, 52, 58, 106, 125, 126, 162, 1..."
5,"[1, 21, 34, 36, 39, 50, 58, 110, 150, 153, 232..."


In [31]:
movie_profile.head()

,title,profile,weights
movieId,,,
1,Toy Story (1995),"[Pixar, pixar, animation, toys, Disney, Tom Ha...","{'Pixar': 0.5233581922752641, 'pixar': 0.43327..."
2,Jumanji (1995),"[Robin Williams, board game, time travel, fant...","{'Robin Williams': 0.695468714308112, 'board g..."
3,Grumpier Old Men (1995),"[moldy, Walter Matthau, Jack Lemmon, fishing, ...","{'moldy': 0.4257623617953107, 'Walter Matthau'..."
4,Waiting to Exhale (1995),"[chick flick, girl movie, characters, interrac...","{'chick flick': 0.6111589664927171, 'girl movi..."
5,Father of the Bride Part II (1995),"[steve martin, pregnancy, childhood classics, ...","{'steve martin': 0.7577224298058415, 'pregnanc..."


# 产生Top-N推荐

In [32]:
user_profile = {}
for uid, mids in watch_record.itertuples():
    record_movie_profile = movie_profile.loc[list(mids)]  # 这里把当前用户看过的电影从movie_profile中找出来
    # 下面需要把这些电影的标签都合并到一块， 然后统计出现的次数, 这里的Counter和reduce用的秒
    counter = collections.Counter(reduce(lambda x, y: list(x) + list(y), record_movie_profile['profile'].values))
    
    # 兴趣词
    interest_words = counter.most_common(50)
    maxcount = interest_words[0][1]
    interest_words = [(w, round(c/maxcount, 4)) for w, c, in interest_words]  # 这里归一化一下
    user_profile[uid] = interest_words

In [33]:
user_profile[1]        # 用户1感兴趣的词

[('Action', 1.0),
 ('Adventure', 0.9556),
 ('Comedy', 0.9222),
 ('Drama', 0.7556),
 ('classic', 0.6222),
 ('Thriller', 0.6222),
 ('Fantasy', 0.5222),
 ('Crime', 0.5),
 ('Children', 0.4667),
 ('Sci-Fi', 0.4444),
 ('action', 0.4),
 ('adventure', 0.3556),
 ('comedy', 0.3556),
 ('fantasy', 0.3444),
 ('Animation', 0.3222),
 ('sci-fi', 0.3222),
 ('funny', 0.3111),
 ('imdb top 250', 0.2889),
 ('Romance', 0.2889),
 ('cult film', 0.2889),
 ('Disney', 0.2778),
 ('quirky', 0.2778),
 ('based on a book', 0.2778),
 ('War', 0.2556),
 ('Musical', 0.2444),
 ('animation', 0.2333),
 ('violence', 0.2333),
 ('surreal', 0.2333),
 ('crime', 0.2222),
 ('great soundtrack', 0.2222),
 ('atmospheric', 0.2111),
 ('war', 0.2111),
 ('humorous', 0.2),
 ('violent', 0.2),
 ('Mystery', 0.2),
 ('thriller', 0.1889),
 ('Horror', 0.1889),
 ('aliens', 0.1889),
 ('talking animals', 0.1778),
 ('music', 0.1778),
 ('Nudity (Topless)', 0.1667),
 ('satire', 0.1667),
 ('musical', 0.1667),
 ('suspense', 0.1556),
 ('tense', 0.1556),


In [34]:
# 为用户产生推荐结果
for uid, interest_words in user_profile.items():
    result_table = {}   # 电影id: [0.2, 0.5]
    for interest_word, interest_weight in interest_words:
        related_movies = inverted_table[interest_word]
        for mid, relate_weight in related_movies:
            _ = result_table.get(mid, [])
            _.append(interest_weight)    #只考虑用户的兴趣程度
            # _.append(related_weight)   # 只考虑兴趣词与电影的关联程度
            # _.append(interest_weight * related_weight)     # 二者都考虑
            result_table.setdefault(mid, _)
    
    rs_result = map(lambda x: (x[0], sum(x[1])), result_table.items()) 
    rs_result = sorted(rs_result, key=lambda x: x[1], reverse=True)[:100]
    print(uid)
    print(rs_result)
    break

NameError: name 'inverted_table' is not defined

# 物品冷启动处理
这里主要包括两个很厉害的技术：
* Word2Vec: 这个可以根据得到电影标签的词向量， 根据这个词向量， 就能够得到tag之间的相似性， 这样就能够根据用户看过的某个电影， 得到这个电影的标签， 然后根据这些标签得到与其近似的标签， 然后得到这些近似标签下的电影对该用户产生推荐
* Doc2Vec：这个可以根据电影的所有标签， 训练一个模型来得到最终电影的影片向量， 根据这个， 就能够直接计算用户看过的某个电影与其他电影的相似性， 然后根据这个相似性给用户推荐最相似的几篇文章。  

这两个的区别体会一下， 下面就看具体的使用

## Word2Vec

In [65]:
from gensim.models import Word2Vec

In [66]:
# 由于前面我们已经得到了每部影片的tags，物品画像里面。 所以这里我们就可以直接建立word2vec模型， 来进行标签的词向量计算
sentences = list(movie_profile["profile"].values)   # 二维列表  每个元素是字符串

In [69]:
# 这里可以直接建立模型
model = Word2Vec(sentences, window=3, min_count=1, iter=20)

In [70]:
# 建立完了之后， 对于某个电影的某个tag， 我们就可以得到与其相似的N个词
words = input("words: ")
ret = model.wv.most_similar(positive=[words], topn=10)   # 找到最相似的n 个词
print(ret)

words: action
[('martial arts', 0.9540493488311768), ('mini coopers', 0.9458670616149902), ('cool premise', 0.9450603127479553), ('stop using useless characters for filler', 0.9410378932952881), ('I AM  THE LAW', 0.936706006526947), ('No humor', 0.9342128038406372), ('not much character development', 0.9305204153060913), ('Cheh Chang', 0.9291208982467651), ('kung fu', 0.9279807209968567), ('SequelTo:Casino Royale[2006]', 0.9274359941482544)]


## Doc2Vec

In [74]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import get_tmpfile

In [72]:
# 建立文档， words就是影片的tags， tags就是影片的id
documents = [TaggedDocument(words, [movie_id]) for movie_id, words in movie_profile["profile"].iteritems()]

In [73]:
documents

[TaggedDocument(words=['Pixar', 'pixar', 'animation', 'toys', 'Disney', 'Tom Hanks', 'computer animation', 'children', 'witty', 'funny', 'animated', 'family', 'friendship', 'clever', 'adventure', 'comedy', 'humorous', 'Cartoon', 'toy', 'time travel', 'buddy movie', 'Tim Allen', 'classic', 'cgi', 'imdb top 250', 'fun', 'fantasy', 'kids', 'John Lasseter', 'Pixar animation', 'Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'], tags=[1]),
 TaggedDocument(words=['Robin Williams', 'board game', 'time travel', 'fantasy', 'animals', 'kid flick', 'bad cgi', 'magic board game', 'scary', 'Kirsten Dunst', 'childhood recaptured', 'monkey', 'not for kids', 'Chris Van Allsburg', 'herds of CGI animals', 'Filmed in BC', 'Joe Johnston', 'Children', 'fiction', 'game', 'time', 'jungle', 'Adaptation of Book', 'see also:Zathura', 'adapted from:book', 'Lebbat', 'thrill', 'For children', 'giant insect', 'new home', 'Adventure', 'Fantasy'], tags=[2]),
 TaggedDocument(words=['moldy', 'Walter Matthau', 'J

In [75]:
# 训练Doc2Vec模型
model = Doc2Vec(documents, vector_size=100, window=3, min_count=1, wordkers=4, epochs=20)

# 模型保存
# fname = get_tmpfile("my_doc2vec_model")
# model.save(fname)

In [76]:
# 获取某个电影的tages
words = movie_profile["profile"].loc[6]
print(words)
# 拿到该影片的Doc2vec向量
inferred_vector = model.infer_vector(words)
sims = model.docvecs.most_similar([inferred_vector], topn=10)
print(sims)

['Al Pacino', 'Robert De Niro', 'realistic action', 'Michael Mann', 'great acting', 'Val Kilmer', 'bank robbery', 'suspense', 'Natalie Portman', 'tense', 'crime', 'atmospheric', 'gunfight', 'dialogue', 'long', 'career criminal', 'philosophy', 'overrated', 'realistic', 'Los Angeles', 'visceral', 'complex characters', 'Bank Heist', 'professionals', 'police', 'too long', 'amazing cast', 'Ashley Judd', 'loner', 'Mary Kircher', 'Action', 'Crime', 'Thriller']
[(6, 0.9746749997138977), (91658, 0.9736606478691101), (1617, 0.9706868529319763), (100306, 0.9698395133018494), (44974, 0.9681257009506226), (1464, 0.9671267867088318), (1248, 0.9664183855056763), (67255, 0.9650761485099792), (288, 0.9642730951309204), (111, 0.9636898636817932)]
